# Compile and Deploy the TFX Pipeline to KFP

This Notebook helps you to compile the **TFX Pipeline** to a **KFP package**. This will creat an **Argo YAML** file in a **.tar.gz** package. We perform the following steps:
1. Build a custom container image that include our modules
2. Compile TFX Pipeline using CLI
3. Deploy the compiled pipeline to KFP 

After you deploy the pipeline, go to KFP UI, create a run, and execute the pipeline:

<img valign="middle" src="imgs/kfp.png" width="800">

## 1. Build Container Image

The pipeline uses a custom docker image, which is a derivative of the [tensorflow/tfx:0.15.0](https://hub.docker.com/r/tensorflow/tfx) image, as a runtime execution environment for the pipeline's components. The same image is also used as a a training image used by **AI Platform Training**.

The custom image modifies the base image by:
 * Downgrading from Tensoflow v2.0 to v1.15 (since AI Platform Prediction is not supporting TF v2.0 yet).
 * Adding the `modules` folder, which includes the **train.py** and **transform.py** code files required by the **Trainer** and **Transform** components, as well as the implementation code for the custom **AccuracyModelValidator** component.


In [ ]:
PROJECT_ID='ksalama-ocado' # Set your GCP project Id
IMAGE_NAME='tfx-image'
TAG='latest'
TFX_IMAGE='gcr.io/{}/{}:{}'.format(PROJECT_ID, IMAGE_NAME, TAG)

!gcloud builds submit --tag $TFX_IMAGE ./ml-pipeline

## 2. Compile TFX Pipeline using CLI

In [ ]:
!tfx pipeline --help

In [ ]:
%%bash 

export PROJECT_ID=$(gcloud config get-value core/project) # Set your GCP project Id

export IMAGE_NAME=tfx-image
export TAG=latest
export TFX_IMAGE=gcr.io/${PROJECT_ID}/${IMAGE_NAME}:${TAG}

export PREFIX=ksalama-mlops-dev # Set your prefix
export NAMESPACE=kfp # Set your namespace
export GCP_REGION=europe-west1 # Set your region
export ZONE=europe-west1-b # Set your zone

export ARTIFACT_STORE_URI=gs://${PREFIX}-artifact-store
export GCS_STAGING_PATH=${ARTIFACT_STORE_URI}/staging
export GKE_CLUSTER_NAME=${PREFIX}-cluster
export DATASET_NAME=sample_datasets # Set your BigQuery Dataset
    
export PIPELINE_NAME=tfx_census_classification
export RUNTIME_VERSION=1.15
export PYTHON_VERSION=3.7

tfx pipeline compile \
    --engine=kubeflow \
    --pipeline_path=ml-pipeline/pipeline.py 

## 3. Deploy the Compiled Pipeline to KFP

In [ ]:
!kfp pipeline --help

In [ ]:
%%bash

export NAMESPACE=kfp # Set your namespac
export PREFIX=ksalama-mlops-dev # Set your prefix
export GKE_CLUSTER_NAME=${PREFIX}-cluster
export ZONE=europe-west1-b # Set your zone
export PIPELINE_NAME=tfx_census_classification

gcloud container clusters get-credentials ${GKE_CLUSTER_NAM}E --zone ${ZONE}
export INVERSE_PROXY_HOSTNAME=$(kubectl describe configmap inverse-proxy-config -n ${NAMESPACE} | grep "googleusercontent.com")

kfp --namespace=${NAMESPACE} --endpoint=${INVERSE_PROXY_HOSTNAME} \
    pipeline upload \
    --pipeline-name=${PIPELINE_NAME} \
    ${PIPELINE_NAME}.tar.gz

## Use the KFP UI to run the deployed pipeline...